In [79]:
import pandas as pd 
import numpy as np
import math
import heapq

In [80]:

def MinkowskiDistance(x,y, p=2): # Distancia Minkowski de x,y vectores en R^n
  dist = 0
  n = len(x)

  for i in range(0,n):
    dist += abs(x[i]-y[i])**p

  dist = dist**(1/p)

  return dist


In [81]:
def frecuencias(arr): # contar las frecuencias de el arreglo con los pares (d(u,x_j), y_j), j=0,1,...,k-1

  count_labels = {} # diccionario con las etiquetas como llaves con sus respectivas frecuencias

  k = len(arr)
      
  for j in range(k): # contar frecuencias
    if arr[j][1] in count_labels:
      count_labels[arr[j][1]] += 1
    else:
      count_labels[arr[j][1]] = 1

  return count_labels 

In [82]:
def eleccion(freq): # Elejir la etiqueta con mayor frecuencia
  # freq es un diccionario cuyos elementos son las frecuencias de la etiqueta que tienen como llave
  keys = list(freq.keys())

  best = keys[0]

  for k in keys: # busqueda de la llave con mas frecuencia
    if freq[k] > freq[best]:
      best = k
  
  return best

In [83]:
def KNN(k, X_prueba, y_prueba, X_test):
  X_prueba = X_prueba.to_numpy()
  y_prueba = y_prueba.to_numpy()
  X_test = X_test.to_numpy()

  n = X_test.shape[0]
  m = X_prueba.shape[0] 

  labeling = []

  for i in range(n):
    u = X_test[i]
    pq = [] 
    heapq.heapify(pq) # Cola de prioridad para guardar los pares (distancia(x,p_i), y_i), donde p_i es el i-esimo vector prototipo con etiqueta y_i

    for j in range(m):
      xj = X_prueba[j]
      dist = MinkowskiDistance(u, xj)
      heapq.heappush(pq,(dist, int(y_prueba[j])))
    
    k_nearest = list(heapq.nsmallest(k, pq)) # Obtener las k menores distancias
    freq = frecuencias(k_nearest)
    labeling.append(eleccion(freq)) 

  return pd.DataFrame(labeling)


In [84]:
def performanceKNN(labels_knn, labels_real, totalLabels):
  # Reporta la matriz de confusion y la exactitud
  # labels_knn es el etiquetado hecho por KNN, labels_real son las etiquetas reales, y totalLabels es el numero total de etiquetas
  # Suponemos que las etiquetas son 1, 2, ..., totalLabels

  labels_knn = labels_knn.to_numpy()
  labels_real = labels_real.to_numpy()

  M = np.zeros((totalLabels, totalLabels))
  n = len(labels_knn)

  for i in range(n):
    M[int(labels_knn[i])-1][int(labels_real[i])-1] += 1

  acc = 0

  for i in range(totalLabels):
    acc += M[i][i]

  acc = acc/n
  M = pd.DataFrame(M, columns= range(1, totalLabels+1), index = range(1, totalLabels+1))

  return M, acc


In [85]:
def split(X, y, p): # funcion para obtener una proporcion p de X de manera aleatoria como conjunto de entrenamiento
  X = X.to_numpy()
  y = y.to_numpy()

  n = len(y)
  d = X.shape[1]
  m = math.floor(p*n) # numero de datos en el conjunto de entrenamiento

  ind = np.random.choice(n, m, replace=False) # muestra aleatoria de tamaño m sin reemplazo de los numeros 0, 1, ..., n-1. 
  # Esta muestra representa los indices que pertenecen al conjunto de entrenamiento

  X_prueba = np.empty((m, d))
  y_prueba = np.empty(m)

  X_test = np.empty((n-m, d))
  y_test = np.empty(n-m)

  next1 = 0
  next2 = 0

  for i in range(n):
    if i in ind:
      X_prueba[next1] = X[i]
      y_prueba[next1] = y[i]
      next1 += 1
    else:
      X_test[next2] = X[i]
      y_test[next2] = y[i]
      next2 += 1

  return pd.DataFrame(X_prueba), pd.DataFrame(y_prueba), pd.DataFrame(X_test), pd.DataFrame(y_test)


In [86]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
X = pd.read_csv(url, header = None)
y = X.iloc[:,0]
X.drop(X.columns[0], axis='columns', inplace=True)
X.columns = range(X.shape[1])
print(X)
print(y)

        0     1     2     3    4     5   ...    7     8      9     10    11    12
0    14.23  1.71  2.43  15.6  127  2.80  ...  0.28  2.29   5.64  1.04  3.92  1065
1    13.20  1.78  2.14  11.2  100  2.65  ...  0.26  1.28   4.38  1.05  3.40  1050
2    13.16  2.36  2.67  18.6  101  2.80  ...  0.30  2.81   5.68  1.03  3.17  1185
3    14.37  1.95  2.50  16.8  113  3.85  ...  0.24  2.18   7.80  0.86  3.45  1480
4    13.24  2.59  2.87  21.0  118  2.80  ...  0.39  1.82   4.32  1.04  2.93   735
..     ...   ...   ...   ...  ...   ...  ...   ...   ...    ...   ...   ...   ...
173  13.71  5.65  2.45  20.5   95  1.68  ...  0.52  1.06   7.70  0.64  1.74   740
174  13.40  3.91  2.48  23.0  102  1.80  ...  0.43  1.41   7.30  0.70  1.56   750
175  13.27  4.28  2.26  20.0  120  1.59  ...  0.43  1.35  10.20  0.59  1.56   835
176  13.17  2.59  2.37  20.0  120  1.65  ...  0.53  1.46   9.30  0.60  1.62   840
177  14.13  4.10  2.74  24.5   96  2.05  ...  0.56  1.35   9.20  0.61  1.60   560

[178 rows x 13 

In [87]:
prop = 0.5 # proporcion de datos de entrenamiento
k = 5
numLabels = 3

In [88]:
X_prueba, y_prueba, X_test, y_test = split(X, y, prop)

In [89]:
labels = KNN(k, X_prueba, y_prueba, X_test)
M, acc = performanceKNN(labels, y_test, numLabels)
print(labels)
print(M)
print(acc)
# Matriz de confusion y exactitud del KNN para el conjunto de prueba

    0
0   1
1   1
2   1
3   1
4   1
.. ..
84  2
85  2
86  2
87  3
88  3

[89 rows x 1 columns]
      1     2     3
1  27.0   2.0   1.0
2   0.0  26.0  18.0
3   1.0   6.0   8.0
0.6853932584269663


In [90]:
labels2 = KNN(k, X_prueba, y_prueba, X_prueba)
M2, acc2 = performanceKNN(labels2, y_prueba, numLabels)
print(labels2)
print(M2)
print(acc2)
# Matriz de confusion y exactitud del KNN para el conjunto de entrenamiento

    0
0   1
1   1
2   1
3   3
4   1
.. ..
84  3
85  2
86  3
87  3
88  2

[89 rows x 1 columns]
      1     2     3
1  27.0   1.0   1.0
2   1.0  33.0   6.0
3   3.0   3.0  14.0
0.8314606741573034
